In [1]:

# Dependencies
import requests
import json
import pandas as pd
import numpy as np
from pprint import pprint
api_format = "json"
from sqlalchemy import create_engine

Part 1- getting data

In [2]:
# for loop for different API 
li=[]
name_q=['games','teams','tips']

for q in name_q:
    s="https://api.squiggle.com.au/?q="+q+";format=json"
    li.append(s)

In [3]:
#p print to check the json data
#pprint(requests.get(li[0]).json())

In [4]:
#convering json data into dataframe 
df_games=pd.DataFrame.from_dict(pd.json_normalize(requests.get(li[0]).json()['games']),orient='columns')

In [5]:
#clean data-Remove columns that we dont needed, rename columns names to make it  clear\n
df_clean=df_games.drop(columns=['localtime', 'date','tz','is_grand_final','complete','is_final','id',
                                'hteamid','ateamid','winnerteamid','updated','round'])
#rename column and assign it into "Games"
Games=df_clean.rename(columns={'abehinds': 'away_behinds', 'hteam': 'home_team','hgoals': 'home_goals','hscore': 'home_score','hgoals': 'home_goals','ascore': 'away_score','agoals': 'away_goals','ateam': 'away_team','hbehinds': 'home_behinds'}) 
Games.info() 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2220 entries, 0 to 2219
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   away_goals    2220 non-null   int64 
 1   year          2220 non-null   int64 
 2   home_behinds  2220 non-null   int64 
 3   home_goals    2220 non-null   int64 
 4   away_behinds  2220 non-null   int64 
 5   away_team     2220 non-null   object
 6   home_team     2220 non-null   object
 7   venue         2220 non-null   object
 8   home_score    2220 non-null   int64 
 9   away_score    2220 non-null   int64 
 10  roundname     2220 non-null   object
 11  winner        2203 non-null   object
dtypes: int64(7), object(5)
memory usage: 208.2+ KB


In [6]:
#getting a list of venue names to get coordinate later 
Games['venue'].unique()

array(['M.C.G.', 'S.C.G.', 'Docklands', 'Carrara', 'Adelaide Oval',
       'Subiaco', 'Sydney Showground', 'Gabba', 'Bellerive Oval',
       'UNSW Canberra Oval', 'University of Tasmania Stadium',
       'Adelaide Arena at Jiangwan Stadium', 'GMHBA Stadium',
       'Traeger Park', 'Marrara Oval', "Cazaly's Stadium", 'Mars Stadium',
       'Optus Stadium', 'Manuka Oval', 'Kardinia Park', 'York Park',
       'Stadium Australia', 'Football Park', 'Blacktown', 'Wellington',
       'Perth Stadium', 'Marvel Stadium', 'Eureka Stadium',
       'Jiangwan Stadium', 'Riverway Stadium'], dtype=object)

In [7]:
#loading players data from csv 
Players=pd.read_csv('stats.csv', encoding="ISO-8859-1") 
#Players.info()
#Players.head()

In [8]:
# getting certain columns for data we need for visualization 
Players=Players[['team','displayName','Disposals','Kicks','Marks','Handballs','Goals','Behinds','Hit Outs','Tackles']]
Players.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 89470 entries, 0 to 89469
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   team         89470 non-null  object
 1   displayName  89470 non-null  object
 2   Disposals    89470 non-null  int64 
 3   Kicks        89470 non-null  int64 
 4   Marks        89470 non-null  int64 
 5   Handballs    89470 non-null  int64 
 6   Goals        89470 non-null  int64 
 7   Behinds      89470 non-null  int64 
 8   Hit Outs     89470 non-null  int64 
 9   Tackles      89470 non-null  int64 
dtypes: int64(8), object(2)
memory usage: 6.8+ MB


In [9]:
#loading venue data 
Venue=pd.read_csv('venue_coord.csv')
Venue.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30 entries, 0 to 29
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Venue       30 non-null     object 
 1   Latitude    29 non-null     float64
 2   Longtitude  29 non-null     float64
dtypes: float64(2), object(1)
memory usage: 848.0+ bytes


Part II: Connecting to SQL 

In [10]:
# connecting 
connection_string = "postgres:iloveyou@localhost:5432/project 2"
engine = create_engine(f'postgresql://{connection_string}')

In [11]:
# transform data
Games.to_sql(name='Games', con=engine, if_exists='replace', index=False)
Players.to_sql(name='Players', con=engine, if_exists='replace', index=False)
Venue.to_sql(name='Venue', con=engine, if_exists='replace', index=False)

In [12]:
# checking table has been created successfully
engine.table_names()

<ipython-input-12-d99326451cb6>:2: SADeprecationWarning: The Engine.table_names() method is deprecated and will be removed in a future release.  Please refer to Inspector.get_table_names(). (deprecated since: 1.4)
  engine.table_names()


['Games', 'Players', 'Venue']

In [13]:
# checking data has been loaded successfully
pd.read_sql_query('select * from "Games"', con=engine).head()

,away_goals,year,home_behinds,home_goals,away_behinds,away_team,home_team,venue,home_score,away_score,roundname,winner
0,20,2017,5,14,12,Richmond,Carlton,M.C.G.,89,132,Round 1,Richmond
1,15,2017,14,12,10,Western Bulldogs,Collingwood,M.C.G.,86,100,Round 1,Western Bulldogs
2,17,2017,10,12,8,Port Adelaide,Sydney,S.C.G.,82,110,Round 1,Port Adelaide
3,18,2017,12,13,12,Melbourne,St Kilda,Docklands,90,120,Round 1,Melbourne
4,15,2017,12,14,8,Brisbane Lions,Gold Coast,Carrara,96,98,Round 1,Brisbane Lions


In [17]:
Games.to_csv (r'Games.csv', index = False, header=True)

In [18]:
Players.to_csv (r'Players.csv', index = False, header=True)